In [19]:
input_files = {
    "ffs": "../static/data/sanfrancisco/ffs_san.csv",
    "speed": "../static/data/sanfrancisco/speed_san.csv",
    "districts": "../static/data/sanfrancisco/Analysis Neighborhoods(b).geojson",
    "district_road_map": "../static/data/sanfrancisco/districts/%s.csv",
}
output_files = {
    "road_tci_date": "../static/data/sanfrancisco/road_tci_date.csv",
    "road_tci_time": "../static/data/sanfrancisco/road_tci_time.csv",
    "city_tci_date": "../static/data/sanfrancisco/city_tci_date.csv",
    "city_tci_time": "../static/data/sanfrancisco/city_tci_time.csv",
    "district_tci_date": "../static/data/sanfrancisco/district_tci_date.csv",
    "district_tci_time": "../static/data/sanfrancisco/district_tci_time.csv",
    "district_tci_date_transposed": "../static/data/sanfrancisco/district_tci_date_transposed.csv",
    "district_tci_time_transposed": "../static/data/sanfrancisco/district_tci_time_transposed.csv",
}

In [20]:
import pandas as pd
import numpy as np
import os, json
from datetime import datetime, timedelta

## FFS

In [21]:
ffs_df = pd.read_csv(input_files['ffs'])
ffs_df

,FFS
0,99.990603
1,55.065793
2,94.382159
3,96.557461
4,47.166686
...,...
4463,97.023134
4464,67.420051
4465,92.849603
4466,75.712457


# 도로별 TCI

- speed(20080517~20080610).csv
    - 행: 5분 간격 시계열
        - 7488개
    - 열: SanFrancisco_edges의 행과 매핑
        - 4468개 (확인필요)
    - 값: 속도(km/h)

In [22]:
start_date = datetime.strptime("2008-05-17", "%Y-%m-%d")

In [23]:
df = pd.read_csv(input_files['speed']).transpose().replace(0, np.nan)
df

,0,1,2,3,4,5,6,7,8,9,...,7016,7017,7018,7019,7020,7021,7022,7023,7024,7025
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# 날짜 단위 데이터프레임 생성
date_df = pd.DataFrame()
date_df.index = df[0].index
for i in range(0, len(df.columns), 12*24):
    cols = df.columns[i:i+12*24]
    sums = df[cols].mean(axis=1)
    colname = start_date + timedelta(days=int(i/(12*24)))

    date_df = pd.concat((date_df, sums.rename(colname)), axis=1)

date_df

,2008-05-17,2008-05-18,2008-05-19,2008-05-20,2008-05-21,2008-05-22,2008-05-23,2008-05-24,2008-05-25,2008-05-26,...,2008-06-01,2008-06-02,2008-06-03,2008-06-04,2008-06-05,2008-06-06,2008-06-07,2008-06-08,2008-06-09,2008-06-10
0,99.990603,106.195309,NaN,NaN,NaN,93.555582,58.168927,NaN,NaN,NaN,...,NaN,86.778021,NaN,NaN,102.965210,NaN,NaN,NaN,NaN,NaN
1,8.357541,NaN,58.404092,NaN,NaN,17.943910,NaN,NaN,NaN,66.516070,...,30.559074,13.409085,NaN,NaN,NaN,10.202780,NaN,NaN,NaN,NaN
2,48.824357,50.031565,86.777742,61.514348,61.934952,78.773444,42.201871,70.608908,37.494939,93.652240,...,47.434076,116.092321,66.100166,38.033621,72.804047,50.743756,73.163894,72.856685,62.130697,NaN
3,94.147401,66.950067,83.493598,79.771106,82.450063,81.547987,87.026850,103.104328,95.517008,74.277057,...,96.743743,91.820906,95.574198,86.636611,88.155476,95.542178,100.308692,90.068414,93.511852,101.564905
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,47.166686,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4463,75.852678,44.747002,72.757176,60.640548,76.150534,89.068450,68.235804,57.382027,78.488982,70.849244,...,94.210022,71.199628,63.996375,82.253467,63.133191,74.213952,70.793947,71.254088,66.452805,60.447080
4464,60.762089,53.776279,70.402195,50.081816,71.549457,57.660560,53.125322,63.776844,77.726507,65.571384,...,51.040246,72.650004,65.053001,67.198314,64.945885,57.927142,61.617524,59.310603,62.062505,NaN
4465,25.273887,69.427038,109.891221,75.252768,53.401202,86.004612,71.231746,68.090875,84.631091,75.376484,...,71.048504,66.083079,87.441198,65.877720,52.069321,31.404979,57.313889,68.303598,46.430845,72.351216
4466,60.064276,54.241152,57.550148,66.515002,52.074041,59.093084,58.311372,73.901383,59.236189,60.870573,...,62.027236,54.819899,62.514630,48.415416,74.165651,64.871825,57.911369,67.838563,64.668299,92.997165


In [25]:
# 시간 단위 데이터프레임 생성
time_df = pd.DataFrame()
time_df.index = df[0].index
for i in range(0, len(df.columns), 12):
    cols = df.columns[i:i+12]
    sums = df[cols].mean(axis=1)
    colname = start_date + timedelta(hours=int(i/12))

    time_df = pd.concat((time_df, sums.rename(colname)), axis=1)

time_df

,2008-05-17 00:00:00,2008-05-17 01:00:00,2008-05-17 02:00:00,2008-05-17 03:00:00,2008-05-17 04:00:00,2008-05-17 05:00:00,2008-05-17 06:00:00,2008-05-17 07:00:00,2008-05-17 08:00:00,2008-05-17 09:00:00,...,2008-06-10 00:00:00,2008-06-10 01:00:00,2008-06-10 02:00:00,2008-06-10 03:00:00,2008-06-10 04:00:00,2008-06-10 05:00:00,2008-06-10 06:00:00,2008-06-10 07:00:00,2008-06-10 08:00:00,2008-06-10 09:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,101.564905,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,60.447080,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,72.351216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,92.997165,NaN,NaN,NaN,NaN,NaN


In [26]:
ffs_date_df = pd.DataFrame(columns=date_df.columns)
for c in ffs_date_df.columns:
    ffs_date_df[c] = ffs_df['FFS']
tci_date_df = pd.DataFrame(date_df.values / ffs_date_df.values, columns=date_df.columns).transpose()
tci_date_df.index.name = "Date"
tci_date_df

,0,1,2,3,4,5,6,7,8,9,...,4458,4459,4460,4461,4462,4463,4464,4465,4466,4467
Date,,,,,,,,,,,,,,,,,,,,,
2008-05-17,1.000000,0.151774,0.517305,0.975040,NaN,NaN,NaN,NaN,NaN,NaN,...,1.118166,1.039301,NaN,0.923925,0.107400,0.781800,0.901247,0.272202,0.793321,NaN
2008-05-18,1.062053,NaN,0.530096,0.693370,NaN,NaN,NaN,NaN,NaN,1.107928,...,1.000000,0.771080,0.650568,0.648080,0.632090,0.461199,0.797630,0.747737,0.716410,NaN
2008-05-19,NaN,1.060624,0.919430,0.864704,NaN,NaN,NaN,NaN,NaN,0.841627,...,NaN,0.880097,1.175614,0.796215,0.366792,0.749895,1.044232,1.183540,0.760115,0.682449
2008-05-20,NaN,NaN,0.651758,0.826152,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.763662,0.390114,0.637176,0.741414,0.625011,0.742833,0.810480,0.878521,0.363634
2008-05-21,NaN,NaN,0.656215,0.853896,NaN,NaN,NaN,NaN,NaN,0.560300,...,NaN,0.829904,1.166064,0.622480,0.815876,0.784870,1.061249,0.575137,0.687787,0.316170
2008-05-22,0.935644,0.325863,0.834622,0.844554,NaN,NaN,NaN,NaN,NaN,0.618282,...,0.317624,0.899325,0.616492,0.741182,0.743092,0.918013,0.855243,0.926279,0.780494,0.687614
2008-05-23,0.581744,NaN,0.447138,0.901296,NaN,NaN,NaN,NaN,NaN,NaN,...,0.656115,0.860913,0.774329,0.708856,0.843774,0.703294,0.787975,0.767173,0.770169,0.895051
2008-05-24,NaN,NaN,0.748117,1.067803,NaN,NaN,NaN,NaN,NaN,NaN,...,0.769207,0.813867,0.830348,0.844315,1.139595,0.591426,0.945963,0.733346,0.976080,NaN
2008-05-25,NaN,NaN,0.397267,0.989225,NaN,NaN,NaN,NaN,NaN,NaN,...,0.868970,0.916763,0.818987,0.852654,0.966517,0.808972,1.152869,0.911486,0.782384,0.906226


In [27]:
ffs_time_df = pd.DataFrame(columns=time_df.columns)
for c in ffs_time_df.columns:
    ffs_time_df[c] = ffs_df['FFS']
tci_time_df = pd.DataFrame(time_df.values / ffs_time_df.values, columns=time_df.columns).transpose()
tci_time_df.index.name = "Time"
tci_time_df

,0,1,2,3,4,5,6,7,8,9,...,4458,4459,4460,4461,4462,4463,4464,4465,4466,4467
Time,,,,,,,,,,,,,,,,,,,,,
2008-05-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-06-10 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-10 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.787714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-10 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
tci_date_df.to_csv(output_files['road_tci_date'])
tci_time_df.to_csv(output_files['road_tci_time'])

# 전체 TCI

In [29]:
city_tci_date_df = pd.DataFrame(tci_date_df.mean(axis=1), columns=["Actual"])
city_tci_date_df

,Actual
Date,
2008-05-17,0.793621
2008-05-18,0.795628
2008-05-19,0.800330
2008-05-20,0.804810
2008-05-21,0.806329
2008-05-22,0.798905
2008-05-23,0.797610
2008-05-24,0.803897
2008-05-25,0.808464


In [30]:
city_tci_time_df = pd.DataFrame(tci_time_df.mean(axis=1), columns=["Actual"])
city_tci_time_df

,Actual
Time,
2008-05-17 00:00:00,NaN
2008-05-17 01:00:00,NaN
2008-05-17 02:00:00,NaN
2008-05-17 03:00:00,NaN
2008-05-17 04:00:00,NaN
...,...
2008-06-10 05:00:00,0.857024
2008-06-10 06:00:00,0.878817
2008-06-10 07:00:00,0.898609


In [31]:
city_tci_date_df.to_csv(output_files['city_tci_date'])
city_tci_time_df.to_csv(output_files['city_tci_time'])

## District별 TCI

In [32]:
with open(input_files['districts']) as file:
    geodata = json.load(file)
district_list = [feature['properties']['nhood'] for feature in geodata['features']]

In [33]:
district_roads = {}
for district in district_list:
    filename = district.replace(" ", "_").replace("/", "_")
    if not os.path.exists(f"../static/data/sanfrancisco/districts/{filename}.csv"):
        continue
    district_roads[district] = pd.read_csv(input_files['district_road_map'] % filename)['Road'].tolist()
    print(max(district_roads[district]))

4343
4324
4392
4317
4421
4456
4319


In [34]:
district_tci_date_map = {}
for district in district_roads.keys():
    district_tci_date_map[district] = tci_date_df.loc[:, district_roads[district]].mean(axis=1)
district_tci_date_df = pd.concat(district_tci_date_map, axis=1)
district_tci_date_df

,South of Market,North Beach,Nob Hill,Mission Bay,Russian Hill,Tenderloin,Financial District/South Beach
Date,,,,,,,
2008-05-17,0.830913,0.773062,0.814164,0.824658,0.798743,0.816992,0.783432
2008-05-18,0.787779,0.777033,0.815204,0.858993,0.798808,0.792716,0.772265
2008-05-19,0.825382,0.789517,0.808093,0.828917,0.806334,0.792944,0.784777
2008-05-20,0.808922,0.830024,0.798131,0.812333,0.820137,0.819898,0.781484
2008-05-21,0.800957,0.819612,0.810479,0.824883,0.813881,0.806871,0.783044
2008-05-22,0.804245,0.818722,0.813389,0.767489,0.817188,0.793001,0.770983
2008-05-23,0.793298,0.792726,0.796155,0.858922,0.804115,0.795353,0.771114
2008-05-24,0.798673,0.808577,0.789720,0.845191,0.809083,0.800457,0.786738
2008-05-25,0.840020,0.760868,0.813302,0.847783,0.811838,0.814693,0.796579


In [35]:
district_tci_time_map = {}
for district in district_roads.keys():
    district_tci_time_map[district] = tci_time_df.loc[:, district_roads[district]].mean(axis=1)
district_tci_time_df = pd.concat(district_tci_time_map, axis=1)
district_tci_time_df

,South of Market,North Beach,Nob Hill,Mission Bay,Russian Hill,Tenderloin,Financial District/South Beach
Time,,,,,,,
2008-05-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-05-17 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2008-06-10 05:00:00,0.832028,0.871757,0.842885,1.041567,0.883704,0.866603,0.824323
2008-06-10 06:00:00,0.837220,0.866149,0.853746,1.249318,0.916007,0.873563,0.843136
2008-06-10 07:00:00,0.811839,0.892320,0.884342,NaN,0.907601,0.960587,0.850974


In [36]:
district_tci_date_df.to_csv(output_files['district_tci_date'])
district_tci_time_df.to_csv(output_files['district_tci_time'])
district_tci_date_df.transpose().to_csv(output_files['district_tci_date_transposed'])
district_tci_time_df.transpose().to_csv(output_files['district_tci_time_transposed'])